#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter Segmentology DV360 Parameters
DV360 funnel analysis using Census data.
 1. Wait for <b>BigQuery->UNDEFINED->UNDEFINED->Census_Pivot</b> to be created.
 1. Join the <a hre='https://groups.google.com/d/forum/starthinker-assets' target='_blank'>StarThinker Assets Group</a> to access the following assets
 1. Copy <a href='https://datastudio.google.com/c/u/0/reporting/4a908845-fdba-4023-9bb7-68666202bafb/page/K15YB/' target='_blank'>DV360 Segmentology</a>. Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to <b>BigQuery->UNDEFINED->(field:recipe_slug}->Census_Pivot</b>.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read': 'user',  # Credentials used for reading data.
  'recipe_project': '',  # Project ID hosting dataset.
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'auth_write': 'service',  # Authorization used for writing data.
  'recipe_name': '',  # Name of report, not needed if ID used.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
  'partners': [],  # DV360 partner id.
  'advertisers': [],  # Comma delimited list of DV360 advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute Segmentology DV360
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'auth': 'user',
      'description': 'Create a dataset for bigquery tables.',
      'hour': [
        4
      ],
      'dataset': {'field': {'description': 'Place where tables will be created in BigQuery.','name': 'recipe_slug','kind': 'string'}}
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'function': 'pearson_significance_test',
      'to': {
        'dataset': {'field': {'description': 'Name of Google BigQuery dataset to create.','name': 'recipe_slug','default': '','order': 4,'kind': 'string'}}
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_ADVERTISER': {
            'values': {'field': {'description': 'Comma delimited list of DV360 advertiser ids.','name': 'advertisers','default': [],'order': 6,'kind': 'integer_list'}}
          },
          'FILTER_PARTNER': {
            'values': {'field': {'description': 'DV360 partner id.','name': 'partners','default': [],'order': 5,'kind': 'integer_list'}}
          }
        },
        'body': {
          'params': {
            'type': 'TYPE_CROSS_PARTNER',
            'groupBys': [
              'FILTER_PARTNER',
              'FILTER_PARTNER_NAME',
              'FILTER_ADVERTISER',
              'FILTER_ADVERTISER_NAME',
              'FILTER_MEDIA_PLAN',
              'FILTER_MEDIA_PLAN_NAME',
              'FILTER_ZIP_POSTAL_CODE'
            ],
            'metrics': [
              'METRIC_BILLABLE_IMPRESSIONS',
              'METRIC_CLICKS',
              'METRIC_TOTAL_CONVERSIONS'
            ]
          },
          'timezoneCode': {'field': {'description': 'Timezone for report dates.','name': 'recipe_timezone','default': 'America/Los_Angeles','kind': 'timezone'}},
          'metadata': {
            'title': {'field': {'description': 'Name of report, not needed if ID used.','order': 3,'name': 'recipe_name','default': '','prefix': 'Segmentology ','kind': 'string'}},
            'dataRange': 'LAST_30_DAYS',
            'format': 'CSV'
          },
          'schedule': {
            'frequency': 'WEEKLY'
          }
        }
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'name': {'field': {'description': 'Name of report, not needed if ID used.','order': 3,'name': 'recipe_name','default': '','prefix': 'Segmentology ','kind': 'string'}}
      },
      'out': {
        'bigquery': {
          'auth': 'user',
          'table': 'DV360_KPI',
          'dataset': {'field': {'description': 'Name of Google BigQuery dataset to create.','name': 'recipe_slug','default': '','order': 4,'kind': 'string'}},
          'schema': [
            {
              'type': 'INTEGER',
              'name': 'Partner_Id',
              'mode': 'REQUIRED'
            },
            {
              'type': 'STRING',
              'name': 'Partner',
              'mode': 'REQUIRED'
            },
            {
              'type': 'INTEGER',
              'name': 'Advertiser_Id',
              'mode': 'REQUIRED'
            },
            {
              'type': 'STRING',
              'name': 'Advertiser',
              'mode': 'REQUIRED'
            },
            {
              'type': 'INTEGER',
              'name': 'Campaign_Id',
              'mode': 'REQUIRED'
            },
            {
              'type': 'STRING',
              'name': 'Campaign',
              'mode': 'REQUIRED'
            },
            {
              'type': 'STRING',
              'name': 'Zip',
              'mode': 'NULLABLE'
            },
            {
              'type': 'FLOAT',
              'name': 'Impressions',
              'mode': 'NULLABLE'
            },
            {
              'type': 'INTEGER',
              'name': 'Clicks',
              'mode': 'NULLABLE'
            },
            {
              'type': 'FLOAT',
              'name': 'Conversions',
              'mode': 'NULLABLE'
            }
          ]
        }
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'from': {
        'query': 'SELECT            Partner_Id,            Partner,            Advertiser_Id,            Advertiser,            Campaign_Id,            Campaign,            Zip,            SAFE_DIVIDE(Impressions, SUM(Impressions) OVER(PARTITION BY Advertiser_Id)) AS Impression_Percent,            SAFE_DIVIDE(Clicks, Impressions) AS Click_Percent,            SAFE_DIVIDE(Conversions, Impressions) AS Conversion_Percent,            Impressions AS Impressions          FROM            `{project}.{dataset}.DV360_KPI`;        ',
        'parameters': {
          'dataset': {'field': {'description': 'Place where tables will be created in BigQuery.','name': 'recipe_slug','kind': 'string'}},
          'project': {'field': {'description': 'Project ID hosting dataset.','name': 'recipe_project','kind': 'string'}}
        },
        'legacy': False
      },
      'to': {
        'view': 'DV360_KPI_Normalized',
        'dataset': {'field': {'description': 'Place where tables will be written in BigQuery.','name': 'recipe_slug','kind': 'string'}}
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'normalize': {
        'census_geography': 'zip_codes',
        'census_span': '5yr',
        'census_year': '2018'
      },
      'to': {
        'type': 'view',
        'dataset': {'field': {'description': 'Name of Google BigQuery dataset to create.','name': 'recipe_slug','default': '','order': 4,'kind': 'string'}}
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'to': {
        'type': 'view',
        'dataset': {'field': {'description': 'Name of Google BigQuery dataset to create.','name': 'recipe_slug','default': '','order': 4,'kind': 'string'}}
      },
      'correlate': {
        'table': 'DV360_KPI_Normalized',
        'correlate': [
          'Impression_Percent',
          'Click_Percent',
          'Conversion_Percent'
        ],
        'sum': [
          'Impressions'
        ],
        'significance': 80,
        'join': 'Zip',
        'dataset': {'field': {'description': 'Name of Google BigQuery dataset to create.','name': 'recipe_slug','default': '','order': 4,'kind': 'string'}},
        'pass': [
          'Partner_Id',
          'Partner',
          'Advertiser_Id',
          'Advertiser',
          'Campaign_Id',
          'Campaign'
        ]
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
